In [197]:
import pandas as pd
from glob import glob

In [198]:
data = pd.read_json('traces/2024-03-13T08:17:24.401727892Z.json', orient='record')

In [199]:
def span_attr(span, key):
    data = pd.DataFrame(span.at['attributes'])
    attr = data[data['key'] == key]
    if not attr.empty:
        return attr['value'].values[0]['stringValue']

In [200]:
def span_event(span, name):
    data = pd.DataFrame(span.at['events'])
    event = data[data['name'] == name]
    if not event.empty:
        return event.squeeze()

In [243]:
span = data[data['name'] == 'CoreAPI.UnixfsAPI.Get']
cid = span_attr(span.squeeze(), 'path').split('/')[-1]
cmd_span = data[data['name'] == 'corehttp.cmdsHandler']
start_nano = cmd_span['startTimeUnixNano'].values[0]
end_nano = cmd_span['endTimeUnixNano'].values[0]
(end_nano - start_nano) / 10**9

1.722143488

In [202]:
cid

'QmQyzGoKyPfixH9shJnYTW1NNd2cV9NWyHohDkbhjSefKC'

In [203]:
other_data = pd.read_json('traces/2024-03-13T08:17:24.401727892Z_other.json', orient='record')

In [204]:
def find_key(span):
    return span.at['name'] == 'Dual.FindProvidersAsync' and span_attr(span, 'key') == cid

find_span = other_data[other_data.apply(find_key, axis=1)]

In [205]:
find_start_nano = int(span_event(find_span.squeeze(), 'started streaming').at['timeUnixNano'])
(find_start_nano - start_nano) / 10**9

1.00123273

In [206]:
find_end_nano = int(span_event(find_span.squeeze(), 'found provider').at['timeUnixNano'])
(find_end_nano - start_nano) / 10**9

1.529068371

In [216]:
span_attr(find_span.squeeze(), 'key')

'QmQyzGoKyPfixH9shJnYTW1NNd2cV9NWyHohDkbhjSefKC'

In [236]:
provider_id = span_attr(span_event(find_span.squeeze(), 'found provider'), 'id')
provider_id

'12D3KooWD7hYatcGYB1nb3AeMN1JFTkVwrePHxDHJEfVNnWVXLSC'

In [235]:
def dial_provider(span):
    # return span.at['name'] == 'KademliaDHT.IpfsDHT.DialPeer' and span_attr(span, 'PeerId') == provider_id
    return span.at['name'] == 'KademliaDHT.IpfsDHT.DialPeer'
for attrs in other_data[other_data.apply(dial_provider, axis=1)]['attributes']:
    print(attrs)

[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWB464XBYQW8LnzzywwGY32oVyYTBysmvfGzC6anmiVmqg'}}, {'key': 'internal.span.format', 'value': {'stringValue': 'otlp'}}]
[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWBB26nVZkX7GqAfcRYpMAv7ABZDzuMtWfCHbXYoAHjijG'}}, {'key': 'internal.span.format', 'value': {'stringValue': 'otlp'}}]
[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWHtyQP4ysH5NJ1Nk4oMGaxBvPzTnFrZLVic3j9tMbHtWk'}}, {'key': 'internal.span.format', 'value': {'stringValue': 'otlp'}}]
[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWSBhhMWajZcaxPNjSLLREgoRVhtLKZqCG53YqtbRjwwKc'}}, {'key': 'internal.span.format', 'value': {'stringValue': 'otlp'}}]
[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWErjJN9TZvCuWRhfVm2G6bcoBgo3WeP3VJ4DMxbZaFGy6'}}, {'key': 'internal.span.format', 'value': {'stringValue': 'otlp'}}]
[{'key': 'PeerID', 'value': {'stringValue': '12D3KooWLWTCeij2EvpxuVXzDsqaNWod6jUhe1egwpNXdDywKF52'}}, {'key': 'internal.span.format', 'value': {'stringValu

In [237]:
other_data['name'].value_counts()

name
KademliaDHT.IpfsDHT.SpawnQuery                   13
KademliaDHT.IpfsDHT.DialPeer                     13
KademliaDHT.IpfsDHT.QueryPeer                    13
KademliaDHT.ProtocolMessenger.GetProviders        6
KademliaDHT.ProviderManager.GetProviders          3
Compose.worker                                    3
Blockservice.blockService.AddBlock                2
KademliaDHT.IpfsDHT.RunQuery                      2
IpfsDHT.FindProvidersAsync                        2
KademliaDHT.IpfsDHT.RunLookupWithFollowup         2
KademliaDHT.IpfsDHT.FindProvidersAsyncRoutine     2
ComposableParallel.FindProvidersAsync             2
KademliaDHT.IpfsDHT.Query.Terminate               1
KademliaDHT.IpfsDHT.Query.Run                     1
KademliaDHT.Dual.worker                           1
Dual.FindProvidersAsync                           1
Name: count, dtype: int64

In [244]:
transfer_start_nano = data[data['name'] == 'Blockservice.Session.GetBlocks']['startTimeUnixNano'].values[0]
(end_nano - transfer_start_nano) / 10**9

0.150677248

In [248]:
data[data['name'] == 'Bitswap.NotifyNewBlocks']['startTimeUnixNano'].min()

1710317845969791744

In [261]:
def report_trace(name):
    data = pd.read_json(f'traces/{name}.json', orient='record')
    span = data[data['name'] == 'CoreAPI.UnixfsAPI.Get']
    cid = span_attr(span.squeeze(), 'path').split('/')[-1]
    # print('cid', cid)
    cmd_span = data[data['name'] == 'corehttp.cmdsHandler']
    start_nano = cmd_span['startTimeUnixNano'].values[0]
    end_nano = cmd_span['endTimeUnixNano'].values[0]
    transfer_start_nano = data[data['name'] == 'Blockservice.Session.GetBlocks']['startTimeUnixNano'].values[0]
    first_block_notify = data[data['name'] == 'Bitswap.NotifyNewBlocks']['startTimeUnixNano'].min()

    other_data = pd.read_json(f'traces/{name}_other.json', orient='record')
    def find_key(span):
        return span.at['name'] == 'Dual.FindProvidersAsync' and span_attr(span, 'key') == cid
    find_span = other_data[other_data.apply(find_key, axis=1)]
    # find_start_nano = int(span_event(find_span.squeeze(), 'started streaming').at['timeUnixNano'])
    find_start_nano = find_span['startTimeUnixNano'].values[0]
    
    # print('find delay', (find_start_nano - start_nano) / 10**9)
    find_delay = (find_start_nano - start_nano) / 10**9
    find_end_nano = int(span_event(find_span.squeeze(), 'found provider').at['timeUnixNano'])
    # print('find', (find_end_nano - find_start_nano) / 10**9)
    find = (find_end_nano - find_start_nano) / 10**9
    # print('from find to first block arrive', (find_end_nano - first_block_notify) / 10**9)
    first_block = (find_end_nano - first_block_notify) / 10**9
    # print('transfer', (end_nano - transfer_start_nano) / 10**9)
    transfer = (end_nano - transfer_start_nano) / 10**9
    # print('total', (end_nano - start_nano) / 10**9)
    total = (end_nano - start_nano) / 10**9
    return pd.Series({'cid': cid, 'find delay': find_delay, 'find': find, 'first block': first_block, 'transfer': transfer, 'total': total})

In [262]:
report_data = pd.DataFrame([
    report_trace(name.split('/')[1].split('_')[0])
    for name in glob('traces/*_other.json')
])
report_data

,cid,find delay,find,first block,transfer,total
0,QmYfvFQRvyDrTcbVVok8ZPGshHEzHbC9cYyhCDFP3TRvud,1.001638,0.732458,-0.032934,0.216827,1.984372
1,QmQyzGoKyPfixH9shJnYTW1NNd2cV9NWyHohDkbhjSefKC,1.001294,0.527880,-0.038995,0.150677,1.722143
2,QmQm44C4QUcRWR68JEwWaubG5KuLpjFpU278AJgexPYHn9,1.001604,0.283456,-0.023157,0.180485,1.490190
3,QmUeeLBzYqbbeAGmwc5t2jhqRgoq1V5e11oNktXwZmR42w,1.002779,2.668487,-0.020380,0.328057,4.021033


In [264]:
report_data.mean(numeric_only=True)

find delay     1.001829
find           1.053070
first block   -0.028867
transfer       0.219012
total          2.304435
dtype: float64